In [1]:
import requests #librerias necesarias
import pandas as pd
import sys 
# sys.exit(1) detiene la ejecución del script INMEDIATAMENTE al encontrar un error
# CRÍTICO (como un archivo faltante o una columna incorrecta), evitando que el programa
# intente continuar y falle después. El número '1' indica que la terminación fue por un error.


apikey = "b4fcada0293085a139072e5e87a365b4"
url = "http://ws.audioscrobbler.com/2.0/"

# lectura de archivo csv
ruta_csv_artistas = '../csv/total_canciones_2018-2022.csv'
# recordamos usar rutas relativas para que funcione en todos los entornos locales

try:
    df_artistas = pd.read_csv(ruta_csv_artistas) #dataframe de artistas desde la ruta de lectura
    
    artistas_con_duplicados = df_artistas['artista'].tolist() #hacemos de 'artista' una lista
    
    artistas_unicos_set = set(artistas_con_duplicados) #con set eliminamos los duplicados
    
    artistas_a_buscar = list(artistas_unicos_set) #aqui ya tenemos la lista limpia con artistas unicos para poder analizar
    
    print(f"✅ Se cargaron {len(artistas_con_duplicados)} registros de artistas (incluidos duplicados).")
    print(f"✨ Se analizarán {len(artistas_a_buscar)} artistas únicos.")

except FileNotFoundError: #los except nos indican que tipo de errores se ha encontrado a la hora de cargar el csv
    print(f"❌ ERROR CRÍTICO: No se encontró el archivo CSV en la ruta '{ruta_csv_artistas}'.")
    sys.exit(1) # El código de salida 1 indica que hubo un error.

except KeyError:
    print(f"❌ ERROR CRÍTICO: La columna 'artista' no se encontró en el archivo CSV.")
    print("Por favor, verifica que el encabezado de la columna en el CSV se llame exactamente 'artista'.")
    sys.exit(1)

except Exception as e:
    print(f"❌ ERROR CRÍTICO: Ocurrió un error inesperado al leer el CSV: {e}")
    sys.exit(1)

# ----------------------------------------------------
# 📌 PUNTO 1: Inicializar la lista para guardar los datos
# ----------------------------------------------------
resultados_api = []
ruta_csv_salida = '../csv/datos_artistas_lastfm.csv' # Define la ruta y nombre del archivo de salida


#bucle para cargar todos los artistas (únicos)
for artista_actual in artistas_a_buscar:
    
    print(f"\n==================================================") #llamadas a la API
    print(f"🔎 Procesando artista: **{artista_actual}**")
    print(f"==================================================")

    # Inicializar un diccionario para los datos de este artista
    datos_artista = {"artista": artista_actual}
    
    #llamada para BIOGRAFIA
    params_info = { #estas lineas no se pueden modificar ya que son necesarias para que nos de la biografia completa
        "method": "artist.getInfo",
        "artist": artista_actual,
        "api_key": apikey,
        "format": "json",
        "autocorrect": 1
    }
    
    try:
        response_info = requests.get(url, params=params_info) 
        datos_info = response_info.json()

        if response_info.status_code == 200 and 'artist' in datos_info: #comprueba que la llamada haya sido exitosa
            info_artista = datos_info['artist']
            
            # --- Extracción y Guardado de datos de BIOGRAFÍA y ESTADÍSTICAS ---
            print("--- BIOGRAFÍA y ESTADÍSTICAS ---")
            biografia_corta = info_artista.get('bio', {}).get('summary', 'No disponible')
            biografia_limpia = biografia_corta.split('<a href')[0].strip()
            
            estadisticas = info_artista.get('stats', {})
            playcount = estadisticas.get('playcount')
            listeners = estadisticas.get('listeners')
            
            # 📌 PUNTO 2: Almacenar datos en el diccionario
            datos_artista['playcount'] = playcount
            datos_artista['listeners'] = listeners
            datos_artista['biografia_resumen'] = biografia_limpia
            
            print(f"Nombre: {info_artista.get('name')}")
            print(f"Biografía (Resumen): {biografia_limpia}")
            print(f"Playcount Total (Last.fm): {playcount}")
            print(f"Número de Oyentes Únicos: {listeners}")
            
        else:
            print(f"❌ Error al obtener info para {artista_actual}. Estado HTTP: {response_info.status_code}")
            datos_artista['playcount'] = None # Si falla, asignamos None para no dejar el campo vacío
            datos_artista['biografia_resumen'] = "API Error"
            
    except requests.RequestException as e:
        print(f"❌ Error de conexión al obtener info para {artista_actual}: {e}")
        datos_artista['playcount'] = None
        datos_artista['biografia_resumen'] = "Connection Error"


    # llamada artistas similares
    similares = {
        "method": "artist.getSimilar",
        "artist": artista_actual,
        "api_key": apikey,
        "format": "json",
        "limit": 5 # Obtener los 5 artistas más similares
    }
    
    # Inicializar la columna de similares
    datos_artista['similares'] = "" # Cadena vacía por defecto
    
    try:
        response_similar = requests.get(url, params=similares)
        datos_similar = response_similar.json()

        if response_similar.status_code == 200 and 'similarartists' in datos_similar:
            artistas_similares = datos_similar['similarartists']['artist']
            
            # Artistas Similares
            print("\n--- ARTISTAS SIMILARES ---")
            if artistas_similares:
                # Crear una lista de nombres de artistas similares y unirlos con coma
                nombres_similares = [similar.get('name') for similar in artistas_similares]
                datos_artista['similares'] = ", ".join(nombres_similares) # 📌 PUNTO 3: Guardar en el diccionario

                for i, similar in enumerate(artistas_similares):
                    print(f"{i+1}. {similar.get('name')} (Similitud: {similar.get('match')})")
            else:
                print("No se encontraron artistas similares.")
        else:
            print(f"❌ Error al obtener similares para {artista_actual}. Estado HTTP: {response_similar.status_code}")
            
    except requests.RequestException as e:
        print(f"❌ Error de conexión al obtener similares para {artista_actual}: {e}")

    # Añadir el diccionario de datos al listado general
    resultados_api.append(datos_artista)


#conversion a csv en el caso de que asi lo queramos
try:
    df_resultados = pd.DataFrame(resultados_api)
    df_resultados.to_csv(ruta_csv_salida, index=False, encoding='utf-8')
    print(f"Datos guardados en '{ruta_csv_salida}'")
    print(f"Columnas guardadas: {list(df_resultados.columns)}")

except Exception as e:
    print(f"\n❌ ERROR al guardar el CSV: {e}")

✅ Se cargaron 1710 registros de artistas (incluidos duplicados).
✨ Se analizarán 537 artistas únicos.

🔎 Procesando artista: **Russell Dickerson**
--- BIOGRAFÍA y ESTADÍSTICAS ---
Nombre: Russell Dickerson
Biografía (Resumen): Multi-Platinum chart-topper and Triple Tigers entertainer Russell Dickerson is no stranger to the grind. The country headliner first broke through by wearing his heart on his sleeve, launching a string of five deeply romantic #1 hits -- “Yours” (3x Platinum), “Blue Tacoma” (2x Platinum), “Love You Like I Used To” (2x Platinum), “Every Little Thing” (Platinum), and “God Gave Me a Girl.” He followed that success with the slinky, soul-infused “She Likes It” (with Jake Scott
Playcount Total (Last.fm): 2580647
Número de Oyentes Únicos: 167661

--- ARTISTAS SIMILARES ---
1. Dustin Lynch (Similitud: 1)
2. Jordan Davis (Similitud: 0.887187)
3. Thomas Rhett (Similitud: 0.772894)
4. Dylan Scott (Similitud: 0.695383)
5. Chris Lane (Similitud: 0.637409)

🔎 Procesando artista